In [1]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster
from folium.features import DivIcon

Loading the data

In [3]:
spacex_csv_file = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df=pd.read_csv(spacex_csv_file)

Setting up dataframe to keep name and coordinates of the launch sites

In [5]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Map setup

In [7]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

Marking the launch sites on the map

In [8]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for i in range (len(launch_sites_df.index)):
    coordinate = [launch_sites_df["Lat"][i], launch_sites_df["Long"][i]]
    circle = folium.Circle(coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup(launch_sites_df["Launch Site"][i]))
    marker = folium.map.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % launch_sites_df["Launch Site"][i],
            )
        )
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map

# Marking the success/failed launches for each site on the map

In [10]:
spacex_df.sample(5)

,Launch Site,Lat,Long,class
22,CCAFS LC-40,28.562302,-80.577356,1
7,CCAFS LC-40,28.562302,-80.577356,0
14,CCAFS LC-40,28.562302,-80.577356,0
10,CCAFS LC-40,28.562302,-80.577356,0
49,CCAFS SLC-40,28.563197,-80.576820,1


In [12]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.sample(5)

,Launch Site,Lat,Long,class,marker_color
8,CCAFS LC-40,28.562302,-80.577356,0,red
55,CCAFS SLC-40,28.563197,-80.576820,0,red
7,CCAFS LC-40,28.562302,-80.577356,0,red
14,CCAFS LC-40,28.562302,-80.577356,0,red
50,CCAFS SLC-40,28.563197,-80.576820,1,green


In [14]:
marker_cluster = MarkerCluster()
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
for index, record in spacex_df.iterrows():
    coordinate = [record['Lat'],record['Long']]
    marker = folium.map.Marker(
        coordinate,
        icon=folium.Icon(color='white',icon_color=record['marker_color'])
        )
    marker_cluster.add_child(marker)

site_map